# PyTorch Deep Neural Networks Training

This notebook trains MLP, LSTM, and GRU models using PyTorch for glucose prediction.

In [ ]:
import os
import sys
import torch
from torch import optim, nn
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

# Add paths for imports
sys.path.append("training")

# Import existing functions
from split_data import load_splits, rescale_data, print_results
from pt_utils_dnn import (
    MLPModel,
    RNNModel,
    EarlyStopper,
    get_dataloader,
    train_model,
    predict_in_batches,
    get_params_count
)

print("Libraries imported successfully")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Configuration and Setup

In [ ]:
# Configuration
config = {
    "output_dir": "outputs",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "seed": 42,
    "batch_size": 4096,
    "dropout": 0.2,
    "activation": "tanh",
    "epochs": 100,
    "lr": 0.01,
    "min_lr": 1e-6,
    "lr_patience": 3,
    "patience": 5,
    "num_layers": 2,
    "models": {
        "mlp": {"hidden_size": 256},
        "lstm": {"hidden_size": 75},
        "gru": {"hidden_size": 86}
    }
}

# Ensure output directory exists
os.makedirs(config["output_dir"], exist_ok=True)

# Set random seed
torch.manual_seed(config["seed"])

print("Configuration:")
for key, value in config.items():
    if key != "models":
        print(f"  {key}: {value}")
print("\nModel configurations:")
for model_name, model_config in config["models"].items():
    print(f"  {model_name}: {model_config}")

## Load and Prepare Data

In [ ]:
print("Loading pre-prepared data splits...")
train_set, val_set, test_set, X_cols, y_cols = load_splits()

print(f"Data loaded successfully:")
print(f"  Train set: {train_set.shape}")
print(f"  Validation set: {val_set.shape}")
print(f"  Test set: {test_set.shape}")
print(f"  Features: {len(X_cols)}")
print(f"  Target: {y_cols}")

## Model Training Functions

In [ ]:
def create_pytorch_model(model_name, model_config, X_cols):
    """Create PyTorch model based on configuration"""
    hidden_size = model_config["hidden_size"]
    
    if model_name == "mlp":
        input_size = len(X_cols)
        model = MLPModel(
            input_size=input_size,
            hidden_sizes=[hidden_size] * config["num_layers"],
            dropout=config["dropout"],
            activation=config["activation"]
        )
    else:  # lstm or gru
        input_size = 1  # For RNN models
        model = RNNModel(
            variant=model_name,
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=config["num_layers"],
            dropout=config["dropout"]
        )
    
    return model


def train_pytorch_model(model_name, model_config):
    """Train a PyTorch model and return results"""
    print(f"\nCreating {model_name.upper()} model...")
    model = create_pytorch_model(model_name, model_config, X_cols)
    
    print(f"\nModel architecture for {model_name.upper()}:")
    get_params_count(model)
    
    # Prepare data loaders
    print(f"\nPreparing data loaders for {model_name.upper()}...")
    train_loader, val_loader = get_dataloader(
        train_set, val_set, X_cols, y_cols, model_name, config["batch_size"]
    )
    
    # Setup training components
    criterion = nn.HuberLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, patience=config["lr_patience"], min_lr=config["min_lr"]
    )
    early_stopper = EarlyStopper(patience=config["patience"])
    
    # Train model
    print(f"\nTraining {model_name.upper()} model...")
    model = train_model(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        early_stopper=early_stopper,
        train_loader=train_loader,
        test_loader=val_loader,
        device=config["device"],
        epochs=config["epochs"],
        output_path=config["output_dir"],
        exp_name=model_name
    )
    
    return model


def evaluate_pytorch_model(model, model_name):
    """Evaluate model and save results"""
    print(f"\nEvaluating {model_name.upper()} model on validation set...")
    
    # Make predictions
    val_set_eval = val_set.copy()
    val_set_eval["y_pred"] = predict_in_batches(
        model, val_set_eval[X_cols], config["device"], model_name
    )
    
    # Prepare results
    val_set_eval = val_set_eval.rename(columns={y_cols[-1]: "target"})
    val_set_eval = rescale_data(val_set_eval, ["target", "y_pred"])
    
    # Select output columns
    output_columns = ["Timestamp", "Patient_ID", "bgClass", "target", "y_pred"]
    results = val_set_eval[output_columns]
    
    # Print evaluation results
    print(f"\nEvaluation results for {model_name.upper()}:")
    print_results(results)
    
    # Save results
    output_file = f"{config['output_dir']}/{model_name}_output.csv"
    results.to_csv(output_file, index=False)
    print(f"Results saved to: {output_file}")
    
    return results

print("Training functions defined successfully")

## Train All Models

In [ ]:
# Train and evaluate each model
results_summary = {}
trained_models = {}

for model_name, model_config in config["models"].items():
    print(f"\n{'='*60}")
    print(f"TRAINING AND EVALUATION - {model_name.upper()}")
    print(f"{'='*60}")
    
    # Train model
    model = train_pytorch_model(model_name, model_config)
    trained_models[model_name] = model
    
    # Evaluate model
    results = evaluate_pytorch_model(model, model_name)
    
    # Store results for summary
    mae = results.apply(lambda row: abs(row['target'] - row['y_pred']), axis=1).mean()
    rmse = ((results['target'] - results['y_pred']) ** 2).mean() ** 0.5
    
    results_summary[model_name] = {
        'Model': model_name.upper(),
        'Hidden Size': model_config['hidden_size'],
        'MAE': round(mae, 4),
        'RMSE': round(rmse, 4),
        'Parameters': sum(p.numel() for p in model.parameters())
    }
    
    print(f"\n{model_name.upper()} training completed!")
    print(f"  MAE: {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  Model saved: {config['output_dir']}/{model_name}.pt")
    print(f"  Results saved: {config['output_dir']}/{model_name}_output.csv")

print(f"\n{'='*60}")
print("ALL MODELS TRAINING COMPLETED")
print(f"{'='*60}")

## Results Summary

In [ ]:
# Create summary DataFrame
summary_data = []
for model_name, metrics in results_summary.items():
    summary_data.append(metrics)

summary_df = pd.DataFrame(summary_data)

print(f"\n{'='*80}")
print("PYTORCH MODELS SUMMARY TABLE:")
print(f"{'='*80}")
print(summary_df.to_string(index=False))

# Save summary
summary_path = f"{config['output_dir']}/pytorch_models_summary.csv"
summary_df.to_csv(summary_path, index=False)
print(f"\nSummary saved to: {summary_path}")

# Find best model
best_model = summary_df.loc[summary_df['MAE'].idxmin()]
print(f"\nBest performing model: {best_model['Model']} (MAE: {best_model['MAE']})")

## Model Comparison

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('default')
sns.set_palette("husl")

# Create comparison plots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# MAE comparison
axes[0].bar(summary_df['Model'], summary_df['MAE'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[0].set_title('Model Performance - MAE Comparison')
axes[0].set_ylabel('MAE')
axes[0].set_xlabel('Model')
for i, v in enumerate(summary_df['MAE']):
    axes[0].text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom')

# Parameters comparison
axes[1].bar(summary_df['Model'], summary_df['Parameters'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[1].set_title('Model Complexity - Parameters Comparison')
axes[1].set_ylabel('Number of Parameters')
axes[1].set_xlabel('Model')
for i, v in enumerate(summary_df['Parameters']):
    axes[1].text(i, v + max(summary_df['Parameters'])*0.01, f'{v:,}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig(f"{config['output_dir']}/pytorch_models_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"\nComparison plot saved to: {config['output_dir']}/pytorch_models_comparison.png")

In [ ]:
print("\n" + "="*60)
print("PYTORCH TRAINING PIPELINE COMPLETED SUCCESSFULLY")
print("="*60)
print(f"\nAll models trained and evaluated:")
for model_name in config["models"].keys():
    print(f"  ✓ {model_name.upper()}")

print(f"\nFiles generated:")
for model_name in config["models"].keys():
    print(f"  - {config['output_dir']}/{model_name}.pt (model weights)")
    print(f"  - {config['output_dir']}/{model_name}_output.csv (predictions)")
print(f"  - {config['output_dir']}/pytorch_models_summary.csv (summary)")
print(f"  - {config['output_dir']}/pytorch_models_comparison.png (comparison plot)")